# SDialog dependencies

In [1]:
# Setup the environment depending on weather we are running in Google Colab or Jupyter Notebook
from IPython import get_ipython


if "google.colab" in str(get_ipython()):
    print("Running on CoLab")

    # Installing Ollama (if you are not planning to use Ollama, you can just comment these lines to speed up the installation)
    !curl -fsSL https://ollama.com/install.sh | sh

    # Installing sdialog
    !git clone https://github.com/qanastek/sdialog.git
    %cd sdialog
    %pip install -e .
    %cd ..
else:
    print("Running in Jupyter Notebook")
    # Little hack to avoid the "OSError: Background processes not supported." error in Jupyter notebooks"
    import os
    get_ipython().system = os.system

Running in Jupyter Notebook


## Locally

Run following commands and then `Restart` your environment.

In [2]:
%%script false --no-raise-error
%pip install -e ..
%pip show sdialog

## Instanciate voices database from HuggingFace HUB

In [3]:
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
voices_libritts = HuggingfaceVoiceDatabase("sdialog/voices-libritts")

/Users/yanislabrak/opt/miniconda3/envs/jsalt/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-10-10 23:10:26] INFO:root:Voice database populated with 2455 voices


Once the database of voice is downloaded and created in the local cache, we will select a voice for a `20` years old `female`.

In [4]:
voices_libritts.get_voice(genre="female", age=20)

{'identifier': 5445,
 'voice': '/Users/yanislabrak/.cache/huggingface/hub/datasets--sdialog--voices-libritts/snapshots/86e8c47af749a36c479e6a24c264c5c8beb3563c/audio/5445_Liz_DeLassus.wav'}

You can also prevent voice to be selected twice, expliciting the parameter `keep_duplicate`:

In [5]:
voices_libritts.get_voice(genre="female", age=20, keep_duplicate=False)

{'identifier': 6683,
 'voice': '/Users/yanislabrak/.cache/huggingface/hub/datasets--sdialog--voices-libritts/snapshots/86e8c47af749a36c479e6a24c264c5c8beb3563c/audio/6683_Julia_Kelley.wav'}

## Custom local voice database

Download voices from our `demo` repository.

In [6]:
import os

# If directory my_custom_voices is not present, download it
if os.path.exists("my_custom_voices"):
    print("my_custom_voices already exists")
else:
    !wget https://raw.githubusercontent.com/qanastek/sdialog/refs/heads/main/misc/audio/my_custom_voices.zip
    !unzip my_custom_voices.zip
    !rm my_custom_voices.zip

my_custom_voices already exists


Once the voices are downloaded in the directory `./my_custom_voices/`, we will create the metadata file that contains the ages, genres and the corresponding voice file for each of the speakers.

In [7]:
from sdialog.audio.voice_database import LocalVoiceDatabase

With CSV metadata file:

In [8]:
voice_database = LocalVoiceDatabase(
    directory_audios="./my_custom_voices/",
    metadata_file="./my_custom_voices/metadata.csv"
)
voice_database.get_voice(genre="female", age=20)

[2025-10-10 23:10:27] INFO:root:Voice database populated with 4 voices


{'identifier': 4,
 'voice': '/Users/yanislabrak/Desktop/HUB/PostJSALT/sdialog/tutorials/my_custom_voices/yanis.wav'}

With TSV metadata file:

In [9]:
voice_database = LocalVoiceDatabase(
    directory_audios="./my_custom_voices/",
    metadata_file="./my_custom_voices/metadata.tsv"
)
voice_database.get_voice(genre="female", age=21)

[2025-10-10 23:10:27] INFO:root:Voice database populated with 4 voices


{'identifier': 3,
 'voice': '/Users/yanislabrak/Desktop/HUB/PostJSALT/sdialog/tutorials/my_custom_voices/thomas.wav'}

With JSON metadata file:

In [10]:
voice_database = LocalVoiceDatabase(
    directory_audios="./my_custom_voices/",
    metadata_file="./my_custom_voices/metadata.json"
)

[2025-10-10 23:10:27] INFO:root:Voice database populated with 4 voices


In [11]:
voice_database.get_voice(genre="female", age=20)

{'identifier': 4,
 'voice': '/Users/yanislabrak/Desktop/HUB/PostJSALT/sdialog/tutorials/my_custom_voices/yanis.wav'}

# Language specific voices

By default all the voices are imported or fetch from/in the database is `english` if no language is specified.

Otherwise, you are able to mention the language you want to work with when you add or get a voice as shown in the following code snippet:

In [12]:
voice_database = LocalVoiceDatabase(
    directory_audios="./my_custom_voices/",
    metadata_file="./my_custom_voices/metadata.json"
)

[2025-10-10 23:10:27] INFO:root:Voice database populated with 4 voices


In [13]:
voice_database.add_voice(
    genre="female",
    age=42,
    identifier="french_female_42",
    path="./my_custom_voices/french_female_42.wav",
    lang="french"
)

Now that a French voice is available in the database, we can retrieve it.

In [14]:
voice_database.get_voice(genre="female", age=20, lang="french")

{'identifier': 'french_female_42',
 'voice': './my_custom_voices/french_female_42.wav'}

But if no voice are available in the targetted language, an error will be thrown:

In [15]:
try:
    voice_database.get_voice(genre="female", age=20, lang="hindi")
except ValueError as e:
    print("Normal error in this case:", e)

Normal error in this case: Language hindi not found in the database
